<a href="https://colab.research.google.com/github/vinayk19/Assignment/blob/master/word_and_countryfinding_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
cd /content/drive/My Drive/AI/basic/data/names_train

/content/drive/My Drive/AI/basic/data/names_train


In [18]:
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

import csv
import gzip

# from name_dataset import NameDataset
# from torch.nn.utils.rnn import pack_padded_sequence, pad_paked_sequence

In [ ]:
# xy1 = pd.read_csv("names_train.csv") #q is it require to convert it(text) into torch
# # xy1.shape
# # name = (xy1.iloc[0,:][0])
# # name
# xy1.describe


In [ ]:
# word = xy1['Adsit'].tolist()
# country = xy1['Czech'].tolist()
# print(len(word))
# print(len(country))
# print(list(dict.fromkeys(country))) # it will provide only unique country name only

In [ ]:
# # filename = '/content/drive/My Drive/AI/basic/data/names_train/names_train.csv.gz' if is_train_set else '/content/drive/My Drive/AI/basic/data/names_train/names_test.csv.gz'
# if is_train_set:
#   filename = '/content/drive/My Drive/AI/basic/data/names_train/names_train.csv.gz'
# else :
#   filename = '/content/drive/My Drive/AI/basic/data/names_train/names_test.csv.gz'

# with open(filename, "rt") as f: # learning if file is zip with file the gzip.open. if only csv file then only open(filename, "rt")
#   reader = csv.reader(f)
#   rows = list(reader)
# print(rows)
# naam = [row[0] for row in rows]
# country = [row[1] for row in rows]

# print(len(naam), naam)
# print(len(country), country)

Tast1 : create database

In [19]:
class Name(Dataset):
  def __init__(self, is_train_set = False):
  # def __init__(self, csv_file):

    if is_train_set:
      filename = '/content/drive/My Drive/AI/basic/data/names_train/names_train.csv'
    else :
      filename = '/content/drive/My Drive/AI/basic/data/names_train/names_test.csv'

    with open(filename, "rt") as f: # learning if file is zip with file the gzip.open. if only csv file then only open(filename, "rt")
      reader = csv.reader(f)
      rows = list(reader)
    # print(rows)
    self.word = [row[0] for row in rows]
    self.country = [row[1] for row in rows]
    # self.xy = pd.read_csv(csv_file)
    # self.word = self.xy['Adsit'].tolist()
    # self.country = self.xy['Czech'].tolist()
    self.country_list = list(dict.fromkeys(self.country))
    
  def __getitem__(self, id):
    return self.word[id], self.country[id]

  def __len__(self):
    return len(self.word)

  def get_country_list(self):
    return self.country_list
  #get the unique country name by its id or lebel
  def get_country_by_id(self, id):
    return self.country_list[id] #@learning callable meance function() but in list it is [] (as list is not callable)
  
  def get_id_by_country(self, country):
    return self.country_list.index(country)

In [20]:
#test database
if __name__ == '__main__':
  database = Name(True) # fasle for train set and true for test set
  print(database.get_country_list())
  print(database.get_id_by_country('French'))
  print(database.get_country_by_id(7))

['Czech', 'German', 'Arabic', 'Japanese', 'Chinese', 'Vietnamese', 'Russian', 'French', 'Irish', 'English', 'Spanish', 'Greek', 'Italian', 'Portuguese', 'Scottish', 'Dutch', 'Korean', 'Polish']
7
French


tast2: Create dataloader

In [21]:
# parameter s and loaderts train loader
# net(N_chars, Hidden_size, N_countries, N_layers)

N_layers = 1
N_countries = len(Name(True).get_country_list())
hidden_size = 20
N_epoch =1
N_chars = 128
batch_size = 256

database_train = Name(True)
train_loader = DataLoader(dataset=database_train, batch_size= batch_size, shuffle=True)

database_test = Name(False)
train_loader = DataLoader(dataset=database_test, batch_size= batch_size, shuffle=True)
print(len(database_train), len(database_test))
print(database_train[2000],database_test[2000])

13374 6700
('Salib', 'Arabic') ('Bakrymov', 'Russian')


In [ ]:
# for val in enumerate(train_loader):
#   print(val[0])

In [106]:
#common funtions 
def make_variable(words, countries):#word to asci charected then padding then pack padd
  words_asc = []
  for word in words:
    words_asc = words_asc + [ascii(word)[0]] #1done now to padding (max len of batch) 

  print("ascii word type", type(words_asc), len(words_asc))
  print("ascii word value",words_asc)
  # learning torch.max works only on tensors not list/tuple no now onwards convert everythings in tensor only
  # ite menans pass the tensors here which are already padded one

  max_len_word = max(len(each) for each in words_asc) # torch.max works in pytorch to change words_asc to tensor
  print("max_len_word value",max_len_word) 
  # padding

  words_padd = [each + [0]*(max_len_word-len(each)) for each in words_asc]
  
  print("words_padd type", type(words_padd), len(words_padd))
  print("words_padd value",words_padd)# till now no tensor 
  #2 padding done 3 pack paadding
  #working # input and lenght should be tensor
  word_packed = torch.nn.utils.rnn.pack_padded_sequence(words_padd, max_len_word)

  print("words_packed type", type(words_packed), len(words_packed))
  print("words_packed value",words_packed)# till now no tensor #passed
  
  Country_ids = [database_train.get_id_by_country(country) for country in countries]
  target = torch.LongTensor(Country_ids)
  print("\ncountry_ids", Country_ids.shape, type(Country_ids), Country_ids)
  print("\nTarget", target.shape, type(target), target)

  return word_packed, target

def ascii(word):
  word_asc = [ord(cha) for cha in word]
  # word_asc = [torch.LongTensor(ord(cha)) for cha in word]

  # return torch.LongTensor(word_asc), len(word_asc)
  return word_asc, len(word_asc)

In [79]:
a = [80, 97, 118, 108, 105, 110, 115, 107, 121], [68, 97, 110, 105, 110], [70, 97, 114, 114, 105, 101, 114], [77, 97, 97, 108, 111, 117, 102]
print(type(a[0]))

<class 'list'>


In [83]:
b = max([len(each) for each in a]) #[torch.FloatTensor(a))
print(b)


9


In [81]:
max(b)

9

In [ ]:
torch.max(b,1)

In [107]:
#final execution funciton
if __name__ == '__main__':
  model = net(N_chars, Hidden_size, N_countries, N_layers)
  optimizer = torch.optim.Adam(model.parameters(), lr=.001)
  criterian = nn.CrossEntropyLoss()

  start = time.time()
  print("Traing for %d epochs ...:" % N_epoch)
  for epoch in range(1, N_epoch+1):
    # Train cycle
    train()
    # Testing cycle
    test()

    #testing several samples
    test("Sung")

Traing for 1 epochs ...:
type make_variable <class 'tuple'> <class 'tuple'>
len  256 20 18 1
ascii word type <class 'list'> 256
ascii word value [[65, 110, 100, 114, 117, 122, 115, 107, 121], [77, 97, 116, 122, 105, 103, 117, 114, 97], [74, 101, 108, 101, 104, 111, 118, 115, 107, 121], [80, 104, 111], [80, 108, 111, 117, 118, 105, 110], [65, 112, 112, 108, 101, 98, 121], [83, 99, 104, 101, 112, 97, 116, 111, 118], [84, 105, 107, 104, 111, 110, 99, 104, 117, 107], [67, 104, 101, 114, 116, 111, 114, 105, 116, 115, 107, 121], [68, 101, 110, 98, 121], [84, 111, 110, 103, 101], [86, 97, 105, 110, 115, 104, 116, 101, 105, 110], [77, 117, 115, 97, 104, 97, 110, 111, 118], [77, 117, 122, 97, 108, 101, 118, 115, 107, 105, 104], [90, 104, 105, 108, 105, 110, 115, 107, 121], [66, 97, 99, 111, 110], [87, 101, 101, 107, 115], [65, 98, 98, 111, 117, 100], [90, 104, 105, 122, 104, 110, 111, 118], [67, 97, 116, 111, 110], [78, 97, 115, 101, 114], [65, 115, 116, 115, 97, 116, 117, 114, 111, 118], [71, 

TypeError: ignored

In [23]:
class net(nn.Module):
  # model = net(N_chars, Hidden_size, N_countries, N_layers)
  def __init__(self, batch_size, hidden_size, N_countries, N_layers=1):
    super(net, self).__init__()
    # print("type", type(batch_size), type(hidden_size), type(N_countries), type(N_layers))
    # print("value are", batch_size, hidden_size, N_countries, N_layers)

    self.vocab_size = batch_size #(it is no of unique words now its a whole batch)
    Embessing_dim = hidden_size # how many input hudden layer can take for each word.
    #lenth of each word ie after padding  no (its acharacter)
    self.num_layer = N_layers
    self.num_bidirection =1
    self.embed = torch.nn.Embedding(batch_size , hidden_size ) # for each word we will get many output = (eq = input to hidden) 
 
    self.grucell = nn.GRU(input_size=hidden_size, hidden_size= hidden_size, num_layers= N_layers )
 
    self.fc = nn.Linear(hidden_size, N_countries)
 
  def forward(self, word_packed):
    print("word_packed type", type(word_packed))
    self.input = word_packed #B*S to S*B
    self.input = input.t() # #B*S to S*B

    #S*B ==> S *B *E
    print("\nTarget", word_packed.shape, type(word_packed), word_packed)
    input = self.embed(self.input) \

    print("\nTarget", output.shape, type(output), output)

    hidden_0 = hidden_init(batch_size)

    output, h_n = self.grucell(input, hidden_0)
    print("output shape", output.shape, "h_n shape", h_n.shape)
    print("output type", type(output), "h_n type", type(h_n))

    logit = self.fc(h_n)

    return logit

  def hiddden_init(self, batch_size):
    h_0 = torch.zeros(self.num_layer*self.num_bidirection, batch_size, self.hidden_size)
    return h_0


In [ ]:
type(len(N_countries))

In [24]:
def train():
  # for epoch in rancge(N_epoch):
  total_loss = 0
  for id, (words, countries) in enumerate(train_loader):
    # make the word proper format ie: words, countryuy to  nos then padding then tensor. 
    print("type make_variable", type(words), type(countries))
    print("len ", batch_size, hidden_size, N_countries, N_layers)
    words, target = make_variable(words, countries)

    output = model(words)
      
    loss = criterion(output, target)
    total_loss += loss.item()
    model.zeros_grad() # learnig we are making gradindier parameter of model not loss 
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
          print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
              time_since(start), epoch,  i *
              len(names), len(train_loader.dataset),
              100. * i * len(names) / len(train_loader.dataset),
              total_loss / i * len(names)))
          
    return total_loss     

In [25]:
def test():
  loss_value = 0
  for id, (words, countries) in enumerate(test_loader):
    words, target = make_variable(words, countries)

    pred = model(words)
    pred_country = pred.max(1) # we got cuntry id now

    # find loss

    if (pred_country == target):
      loss_value = loss_value
    else:
      loss_value = loss_value +1

  return loss_value

In [ ]:
""

In [26]:
def pred(word):
  output = model(word)
  country = train_database.get_country_by_id(output.max(1))
  return country

In [ ]:
# #test loader
# database_test = Name('/content/drive/My Drive/AI/basic/data/names_train/names_test.csv')
# test_loader = DataLoader(dataset=database_test, batch_size= BS, shuffle=True)

working on asc and tranposzing .t() is it tup[le , tensor or what then next action
1. remove asc use direct embedding
2. use asc only noo emabeddinbg as asx itself is embedding
by my case ASC is streamline word length as well. so try to keep asx alive for time being untill batch size is 10

task3: model: embedding, rnn, linear layer 
task4: optimizer
task5: training
taask6: test

In [ ]:
# class net(torch.nn.Module): #learning we can get fuction created inside database (it can be train or test)
#   def __init__(self):
#     super(net,self).__init__()
#     self.word_len = 18 # seq chenged from 15 to 18 for test
#     self.vocab_size = self.word_len # no of character per word
#     self.embedding_dim = 6 # how many diamention it wants
#     self.input_size = self.embedding_dim #6
#     self.hiddenO = self.input_size #6 dim
#     self.hidden_size = self.embedding_dim #6
#     self.batch = 1
#     self.num_layer =1
#     self.num_class = 18 #num of contries

#     #embedding  @learning use tensor conversion al last meand at GRU ot RNN time not beofore that as arugment of other function (like embed) are not tensors
#     self.embed = nn.Embedding(num_embeddings = self.vocab_size, embedding_dim = self.embedding_dim) # num_enn = input size word lenth and embedd is output size ie hidden size
#     #gru    
#     self.grucell = nn.GRU(input_size = self.input_size, hidden_size = self.hidden_size, num_layers =self.num_layer, batch_first = True)
#     #fc
#     self.fc = nn.Linear(in_features= self.embedding_dim, out_features= self.num_class)
  
#   def forward(self, word, country): # 10 word and 10 country
#     # word, contry = data
#     print("word", word) # word type is tuple of B*S
#     #converting work in to integer ASCII for input and outpuut ord # learning asc is not required as embedding can take words directly
     
#     word_ascii, w_len = self.asc(word, self.word_len) #in: BS * word and out: BS * ASCII with padding
#     print("\n word_ascii", word_ascii,  "\n w_len", w_len)
#       # challenge we need some padding. keep every word as 18 character long.
#       #learning embedding needs word(s) * BS  ir S*B not B*S
      
#     word_ascii = list(zip(*word_ascii)) # in B*S ot S*B
#     # print("word ascii ", word_ascii)
#     word_ascii = torch.LongTensor(word_ascii)

#     #@ learning so word equalization you shoiuld do after embedfding  not before that. otherwise embedding will not ass anything

#     word_embed = self.embed(word_ascii) #embedding e extra in matrix out : BS* word_len*E
#       #rnn in and out rnn_in BS*Seq*inu_D(e) and rnn_out BS*Seq*hiddenO
#       # input = B*Seq_len*input, hidden = num_lauyer* batch,hidden_size
#     input = word_embed # B*S*I
#     hidden = self.hidden_init()
#     out, h_n = self.grucell(input, hidden) #out B*S*hidden [1 10 6] , h_n = num_layerxBxHidden [1 1 6]

#     #FC linear layer in Seq to out no of classor contry(-1, nod of class or country)
#     out = self.fc(h_n) #in [1 1 6] out [1 1 18]
#     out = out.view(-1, self.num_class)
#     # out - out.t()
#     country_ids = self.country2tensor(country) # 18 contoury with each 15 word
#     # country = country.t()#view(-1)
#     # print("country ids", country_ids.shape)
#     # country = country.squeeze_()
#     # out = out.squeeze_()
#     return out, country_ids

#     #output output of DC 
#   def hidden_init(self):
#     hid = torch.zeros(self.num_layer, self.batch, self.hiddenO)
#     return hid
#   def country2tensor(self, countries):
#     # print("contries size", len(countries))
#     country_ids = [database.get_id_by_country(each_country) for each_country in countries]
#     # print("contries ids", len(country_ids))
#     return torch.LongTensor(country_ids)

#   def asc(self, words, word_len):
#     word_len = word_len
#     sorks3 =  []
#     # sorks3 = torch.zeros[len(words)]
#     for word in words:
#       # # for cha in word:
#       # sorks = [ord(cha) for cha in word]
#       # sorks2 = [sorks + [0]*(word_len-len(sorks))] 
#       # sorks3 = Variable(torch.LongTensor([sorks2]))
#       # sorks4 = sorks4.append(sorks3)
      
#       # sorks3.append(Variable(torch.LongTensor([[ord(cha) for cha in word] + [0]*(word_len-len([ord(cha) for cha in word]))])))
#       sorks3.append([[ord(cha) for cha in word] + [0]*(word_len-len([ord(cha) for cha in word]))])
#       # i = i+1
#     return sorks3, len(sorks3) 


# model = net()

In [ ]:
# class net(torch.nn.Module): #learning we can get fuction created inside database (it can be train or test)
#   def __init__(self):
#     super(net,self).__init__()
#     self.word_len = 18 # seq chenged from 15 to 18 for test
#     self.vocab_size = self.word_len # no of character per word
#     self.embedding_dim = 6 # how many diamention it wants
#     self.input_size = self.embedding_dim #6
#     self.hiddenO = self.input_size #6 dim
#     self.hidden_size = self.embedding_dim #6
#     self.batch = 1
#     self.num_layer =1
#     self.num_class = 18 #num of contries

#     #embedding  @learning use tensor conversion al last meand at GRU ot RNN time not beofore that as arugment of other function (like embed) are not tensors
#     self.embed = nn.Embedding(num_embeddings = self.vocab_size, embedding_dim = self.embedding_dim) # num_enn = input size word lenth and embedd is output size ie hidden size
#     #gru    
#     self.grucell = nn.GRU(input_size = self.input_size, hidden_size = self.hidden_size, num_layers =self.num_layer, batch_first = True)
#     #fc
#     self.fc = nn.Linear(in_features= self.embedding_dim, out_features= self.num_class)
  
#   def forward(self, word, country): # 10 word and 10 country
#     # word, contry = data
#     print("word", word) # word type is tuple of B*S

#     # challenge we need some padding. keep every word as 18 character long.
#     #learning embedding needs word(s) * BS  ir S*B not B*S
      
#     word_ascii = list(zip(*word_ascii)) # in B*S ot S*B
#     # print("word ascii ", word_ascii)
#     word_ascii = torch.LongTensor(word_ascii)

#     #@ learning so word equalization you shoiuld do after embedfding  not before that. otherwise embedding will not ass anything
    
#     word_embed = self.embed(word_ascii) #embedding e extra in matrix out : BS* word_len*E
#       #rnn in and out rnn_in BS*Seq*inu_D(e) and rnn_out BS*Seq*hiddenO
#       # input = B*Seq_len*input, hidden = num_lauyer* batch,hidden_size
#     input = word_embed # B*S*I
#     hidden = self.hidden_init()
#     out, h_n = self.grucell(input, hidden) #out B*S*hidden [1 10 6] , h_n = num_layerxBxHidden [1 1 6]

#     #FC linear layer in Seq to out no of classor contry(-1, nod of class or country)
#     out = self.fc(h_n) #in [1 1 6] out [1 1 18]
#     out = out.view(-1, self.num_class)
#     # out - out.t()
#     country_ids = self.country2tensor(country) # 18 contoury with each 15 word
#     # country = country.t()#view(-1)
#     # print("country ids", country_ids.shape)
#     # country = country.squeeze_()
#     # out = out.squeeze_()
#     return out, country_ids

#     #output output of DC 
#   def hidden_init(self):
#     hid = torch.zeros(self.num_layer, self.batch, self.hiddenO)
#     return hid
#   def country2tensor(self, countries):
#     # print("contries size", len(countries))
#     country_ids = [database.get_id_by_country(each_country) for each_country in countries]
#     # print("contries ids", len(country_ids))
#     return torch.LongTensor(country_ids)

#   def asc(self, words, word_len):
#     word_len = word_len
#     sorks3 =  []
#     # sorks3 = torch.zeros[len(words)]
#     for word in words:
#       # # for cha in word:
#       # sorks = [ord(cha) for cha in word]
#       # sorks2 = [sorks + [0]*(word_len-len(sorks))] 
#       # sorks3 = Variable(torch.LongTensor([sorks2]))
#       # sorks4 = sorks4.append(sorks3)
      
#       # sorks3.append(Variable(torch.LongTensor([[ord(cha) for cha in word] + [0]*(word_len-len([ord(cha) for cha in word]))])))
#       sorks3.append([[ord(cha) for cha in word] + [0]*(word_len-len([ord(cha) for cha in word]))])
#       # i = i+1
#     return sorks3, len(sorks3) 

# model = net()

Process contry differently as each contry will have unique no (one number not as matrix as word (N * no of class) will be comparing or predicting the perticular class. so its a one idea to keep perticular class as one number rather than keeping it as one class as array.

TODO each contry as one unique no..

In [ ]:
# Criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr = .0001)

Trainig

https://stackoverflow.com/questions/49206550/pytorch-error-multi-target-not-supported-in-crossentropyloss

In [ ]:
# for epoch in range(1):
#   total_loss = 0 
#   # batch_id = 0 
#   for batch_idx, (word, country) in enumerate(train_loader): #@ learning loader is sending tuple on each count
#     print("word database", type(word), "countey database", type(country))
#     out, country = model(word, country)
#     print("out shape", out.shape, "countey shape", country.shape)
#     optimizer.zero_grad()
#     loss = Criterion(out, country)
#     loss.backward()
#     optimizer.step()
#     if batch_idx % 500  == 0:
#         print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
#                 epoch, batch_idx * len(word), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))
    

In [ ]:
# n= ["name"]
# # n[1]
# # ord(n[1])
# print(asc(n,10)[0].size())
# a, b = asc(n,10)
# embed=nn.Embedding(200,3)
# emb2 = embed(a)

In [ ]:
# print(emb2.shape)

In [ ]:
  # sorks= []
  # i=0
  # def asc(words, word_len):
  #   word_len = word_len

  #   for word in words:
  #     # for cha in word:
  #     sorks = [ord(cha) for cha in word]
  #     sorks2 = sorks + [0]*(word_len-len(sorks)) 
  #     sorks3 = Variable(torch.LongTensor([sorks2]))
  #   return sorks3, len(sorks2)

In [ ]:
# n[1]

In [ ]:
# n1 =asc(n)

In [ ]:
# n2 = concat(n1, zeros[2])

In [ ]:
# like_zero[2]